# Load and prepare data
By now we have run the airflow dags to ingest a period of a few months worth of API data. 

Let's now load in the data from our postgres database and do some preprocessing

In [ ]:
%load_ext autoreload
%autoreload 2

In [81]:
import pandas as pd
from sqlalchemy import create_engine, text
import seaborn as sns
from pathlib import Path
from dotenv import find_dotenv
import os

In [82]:
REPO_ROOT = Path(find_dotenv()).parent
CONNECTION_STRING = os.environ["DB_CONNECTION_STRING"]

: 

In [55]:
# Create Postgres connection object
engine = create_engine(CONNECTION_STRING)

# Transformation pipeline

## Clean, resample, transform & merge tables

The below section will be tidied up and refactored into a python module.

See `src/house_climate/data/preprocess.py`

I'm keeping this interactive notebook version here as it's easier to experiment with, but all downstream tasks will use the dataset generated by the above python module version 

## "Interior" table

In [56]:
with engine.begin() as conn:
    interior = pd.read_sql("SELECT * FROM interior WHERE extracted_date BETWEEN '2022-01-01' AND '2023-01-01'", con=engine)

interior.sort_values(['timestamp', 'zone_id'])

,extracted_date,timestamp,zone_id,humidity,humidity_unit,temperature,temperature_unit
2783,2022-01-01,2022-01-01 00:00:00+00:00,1,0.500,percentage,20.00,celsius
721,2022-01-01,2022-01-01 00:00:00+00:00,2,0.500,percentage,20.00,celsius
2881,2022-01-01,2022-01-01 00:00:00+00:00,6,0.500,percentage,20.00,celsius
3273,2022-01-01,2022-01-01 00:00:00+00:00,7,0.500,percentage,20.00,celsius
3077,2022-01-01,2022-01-01 00:00:00+00:00,9,0.500,percentage,20.00,celsius
...,...,...,...,...,...,...,...
286071,2022-12-31,2023-01-01 00:00:00+00:00,12,0.607,percentage,21.87,celsius
7,2023-01-01,2023-01-01 00:00:00+00:00,16,0.631,percentage,21.50,celsius
285777,2022-12-31,2023-01-01 00:00:00+00:00,16,0.631,percentage,21.50,celsius
17,2023-01-01,2023-01-01 00:00:00+00:00,17,0.589,percentage,19.41,celsius


In [57]:
interior.zone_id.value_counts()

10    36131
9     36130
2     36128
6     36128
1     36125
7     36125
12    36124
17    17133
16    17131
Name: zone_id, dtype: int64

In [58]:
interior.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287155 entries, 0 to 287154
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype              
---  ------            --------------   -----              
 0   extracted_date    287155 non-null  object             
 1   timestamp         287155 non-null  datetime64[ns, UTC]
 2   zone_id           287155 non-null  int64              
 3   humidity          287155 non-null  float64            
 4   humidity_unit     287155 non-null  object             
 5   temperature       287155 non-null  float64            
 6   temperature_unit  287155 non-null  object             
dtypes: datetime64[ns, UTC](1), float64(2), int64(1), object(3)
memory usage: 15.3+ MB


In [59]:
# Clean up duplicates
def clean_interior(interior):# Clean up duplicates
    interior = (
        interior
        .drop_duplicates('timestamp', keep='first')
    )
    return interior

(
    interior
    .groupby('zone_id').apply(clean_interior)
    .set_index('timestamp')
)

,extracted_date,zone_id,humidity,humidity_unit,temperature,temperature_unit
timestamp,,,,,,
2022-12-31 23:45:00+00:00,2023-01-01,1,0.570,percentage,23.36,celsius
2023-01-01 00:00:00+00:00,2023-01-01,1,0.571,percentage,23.14,celsius
2022-01-16 23:45:00+00:00,2022-01-17,1,0.500,percentage,20.00,celsius
2022-01-17 00:00:00+00:00,2022-01-17,1,0.500,percentage,20.00,celsius
2022-01-17 00:15:00+00:00,2022-01-17,1,0.500,percentage,20.00,celsius
...,...,...,...,...,...,...
2022-07-12 21:30:00+00:00,2022-07-12,17,0.495,percentage,23.59,celsius
2022-07-12 21:45:00+00:00,2022-07-12,17,0.497,percentage,23.70,celsius
2022-07-12 22:00:00+00:00,2022-07-12,17,0.498,percentage,23.75,celsius


In [61]:
# Resample timeseries to 1minute intervals
# Numerical columns get linear interpolated from the existing 15minute intervals
# Non numerics get forward filled

def upsample_interior(interior):
    interpolated_num_cols = (
        interior
        .loc[:, ["humidity", "temperature"]]
        .resample('1T')
        .interpolate('linear')
    )

    interpolated_other_cols = (
        interior
        .loc[:, ["zone_id", "humidity_unit", "temperature_unit"]]
    )

    new_interior = (
        pd.merge(
            left=interpolated_num_cols, 
            right=interpolated_other_cols, 
            on='timestamp', 
            how='left'
        )
        .ffill()
        .astype({"zone_id": "Int8"})
    )
    return new_interior


interior_resampled = (
    interior
    .groupby('zone_id').apply(clean_interior)
    .set_index('timestamp')
    .groupby('zone_id').apply(upsample_interior)
    .ffill()
    .reset_index(0, drop=True)
    .sort_values(by=['timestamp', 'zone_id'])
)

interior_resampled

,humidity,temperature,zone_id,humidity_unit,temperature_unit
timestamp,,,,,
2022-01-01 00:00:00+00:00,0.500,20.00,1,percentage,celsius
2022-01-01 00:00:00+00:00,0.500,20.00,2,percentage,celsius
2022-01-01 00:00:00+00:00,0.500,20.00,6,percentage,celsius
2022-01-01 00:00:00+00:00,0.500,20.00,7,percentage,celsius
2022-01-01 00:00:00+00:00,0.500,20.00,9,percentage,celsius
...,...,...,...,...,...
2023-01-01 00:00:00+00:00,0.585,20.36,9,percentage,celsius
2023-01-01 00:00:00+00:00,0.597,22.44,10,percentage,celsius
2023-01-01 00:00:00+00:00,0.607,21.87,12,percentage,celsius


---

## "call_for_heat" data

In [69]:
with engine.begin() as conn:
    heat = pd.read_sql("SELECT * FROM call_for_heat WHERE extracted_date BETWEEN '2022-01-01' AND '2023-01-01'", con=engine)

heat

,extracted_date,zone_id,t_start,t_end,value
0,2023-01-01,2,2022-12-31 23:45:00+00:00,2023-01-01 18:58:29.597000+00:00,NONE
1,2023-01-01,2,2023-01-01 18:58:29.597000+00:00,2023-01-01 18:59:04.512000+00:00,HIGH
2,2023-01-01,2,2023-01-01 18:59:04.512000+00:00,2023-01-01 19:13:06.147000+00:00,NONE
3,2023-01-01,2,2023-01-01 19:13:06.147000+00:00,2023-01-01 19:28:16.927000+00:00,HIGH
4,2023-01-01,2,2023-01-01 19:28:16.927000+00:00,2023-01-02 00:15:00+00:00,NONE


In [72]:
heat_preprocessed = (
    heat
    .sort_values(by=['t_start', 'zone_id'])
    .astype({"zone_id": "Int8"})
)
heat_preprocessed

,extracted_date,zone_id,t_start,t_end,value
183,2022-01-01,1,2021-12-31 23:45:00+00:00,2022-01-02 00:15:00+00:00,NONE
182,2022-01-01,2,2021-12-31 23:45:00+00:00,2022-01-02 00:15:00+00:00,NONE
184,2022-01-01,6,2021-12-31 23:45:00+00:00,2022-01-02 00:15:00+00:00,NONE
194,2022-01-01,7,2021-12-31 23:45:00+00:00,2022-01-02 00:15:00+00:00,NONE
186,2022-01-01,9,2021-12-31 23:45:00+00:00,2022-01-02 00:15:00+00:00,NONE


---

In [73]:
# Merge both tables to get one unified table of 1-minute intervals for temp, humidity and heating status
def filter_by_time_ranges(df):
    return df.loc[lambda x: (x.index >= x.t_start) & (x.index <= x.t_end)]


interior_and_heat = (
    pd.merge_asof(
        interior_resampled, heat_preprocessed, 
        by='zone_id', left_index=True, 
        right_on='t_start', 
        direction='backward'
    )
    .groupby('zone_id').apply(filter_by_time_ranges)
    .reset_index(0, drop=True)
    .sort_values(by=['timestamp', 'zone_id'])
    .loc[:, ['humidity', 'temperature', 'zone_id', 'value']]
)

interior_and_heat


(4177530, 4)


,humidity,temperature,zone_id,value
timestamp,,,,
2022-01-01 00:00:00+00:00,0.500,20.00,1,NONE
2022-01-01 00:00:00+00:00,0.500,20.00,2,NONE
2022-01-01 00:00:00+00:00,0.500,20.00,6,NONE
2022-01-01 00:00:00+00:00,0.500,20.00,7,NONE
2022-01-01 00:00:00+00:00,0.500,20.00,9,NONE
...,...,...,...,...
2023-01-01 00:00:00+00:00,0.585,20.36,9,HIGH
2023-01-01 00:00:00+00:00,0.597,22.44,10,NONE
2023-01-01 00:00:00+00:00,0.607,21.87,12,NONE


---

## "weather" table

In [74]:
# Load in weather table
with engine.begin() as conn:
    weather = pd.read_sql("SELECT * FROM weather WHERE extracted_date BETWEEN '2022-01-01' AND '2023-01-01'", con=engine)

weather

,extracted_date,t_start,t_end,zone_id,state,temp_celsius
0,2023-01-01,2022-12-31 23:45:00+00:00,2022-12-31 23:58:42.233000+00:00,2,NIGHT_CLOUDY,1.31
1,2023-01-01,2022-12-31 23:58:42.233000+00:00,2023-01-01 00:13:42.531000+00:00,2,NIGHT_CLOUDY,1.31
2,2023-01-01,2023-01-01 00:13:42.531000+00:00,2023-01-01 00:28:42.794000+00:00,2,NIGHT_CLOUDY,1.19
3,2023-01-01,2023-01-01 00:28:42.794000+00:00,2023-01-01 00:43:42.937000+00:00,2,NIGHT_CLOUDY,1.19
4,2023-01-01,2023-01-01 00:43:42.937000+00:00,2023-01-01 00:58:42.679000+00:00,2,NIGHT_CLOUDY,1.19


In [75]:
weather_preprocessed = (
    weather
    .sort_values(by=['t_start', 'zone_id'])
    .rename(columns={'temp_celsius': 'exterior_temp', 'state': 'weather'})
    .astype({"zone_id": "Int8"})
)
weather_preprocessed

,extracted_date,t_start,t_end,zone_id,weather,exterior_temp
899,2022-01-01,2021-12-31 23:45:00+00:00,2022-01-02 00:15:00+00:00,1,UNKNOWN,NaN
898,2022-01-01,2021-12-31 23:45:00+00:00,2022-01-02 00:15:00+00:00,2,UNKNOWN,NaN
900,2022-01-01,2021-12-31 23:45:00+00:00,2022-01-02 00:15:00+00:00,6,UNKNOWN,NaN
904,2022-01-01,2021-12-31 23:45:00+00:00,2022-01-02 00:15:00+00:00,7,UNKNOWN,NaN
902,2022-01-01,2021-12-31 23:45:00+00:00,2022-01-02 00:15:00+00:00,9,UNKNOWN,NaN
...,...,...,...,...,...,...
597,2023-01-01,2023-01-02 00:13:42.416000+00:00,2023-01-02 00:15:00+00:00,9,NIGHT_CLOUDY,0.4
497,2023-01-01,2023-01-02 00:13:42.416000+00:00,2023-01-02 00:15:00+00:00,10,NIGHT_CLOUDY,0.4
697,2023-01-01,2023-01-02 00:13:42.416000+00:00,2023-01-02 00:15:00+00:00,12,NIGHT_CLOUDY,0.4
397,2023-01-01,2023-01-02 00:13:42.416000+00:00,2023-01-02 00:15:00+00:00,16,NIGHT_CLOUDY,0.4


In [76]:
int_heat_weather = (
    pd.merge_asof(
        interior_and_heat, weather_preprocessed, 
        by='zone_id', 
        left_index=True, right_on='t_start', 
        direction='backward',
    )
    .groupby('zone_id').apply(filter_by_time_ranges)
    .reset_index(0, drop=True)
    .sort_values(by=['timestamp', 'zone_id'])
    .loc[:, ['humidity', 'temperature', 'zone_id', 'value', 'weather', 'exterior_temp', 'extracted_date']]
)

int_heat_weather

(4177408, 7)


,humidity,temperature,zone_id,value,weather,exterior_temp,extracted_date
timestamp,,,,,,,
2022-01-01 00:00:00+00:00,0.500,20.00,1,NONE,UNKNOWN,NaN,2022-01-01
2022-01-01 00:00:00+00:00,0.500,20.00,2,NONE,UNKNOWN,NaN,2022-01-01
2022-01-01 00:00:00+00:00,0.500,20.00,6,NONE,UNKNOWN,NaN,2022-01-01
2022-01-01 00:00:00+00:00,0.500,20.00,7,NONE,UNKNOWN,NaN,2022-01-01
2022-01-01 00:00:00+00:00,0.500,20.00,9,NONE,UNKNOWN,NaN,2022-01-01
...,...,...,...,...,...,...,...
2023-01-01 00:00:00+00:00,0.585,20.36,9,HIGH,UNKNOWN,NaN,2022-12-31
2023-01-01 00:00:00+00:00,0.597,22.44,10,NONE,UNKNOWN,NaN,2022-12-31
2023-01-01 00:00:00+00:00,0.607,21.87,12,NONE,UNKNOWN,NaN,2022-12-31


---

## "days" table

In [78]:
# Load in days table
with engine.begin() as conn:
    days = pd.read_sql("SELECT * FROM days WHERE extracted_date BETWEEN '2022-01-01' AND '2023-01-01'", con=engine)

days

,extracted_date,t_start,t_end,zone_id,zone_type,zone_name,hours_in_day,historic_data,zone_metadata
0,2022-01-01,2021-12-31 23:45:00+00:00,2022-01-02 00:15:00+00:00,9,HEATING,Office,24,{'callForHeat': {'dataIntervals': [{'from': '2...,"{'dateCreated': '2020-12-02T20:05:04.960Z', 'd..."
1,2022-01-01,2021-12-31 23:45:00+00:00,2022-01-02 00:15:00+00:00,12,HEATING,Upstairs hallway,24,{'callForHeat': {'dataIntervals': [{'from': '2...,"{'dateCreated': '2020-12-25T09:39:10.393Z', 'd..."
2,2023-01-01,2022-12-31 23:45:00+00:00,2023-01-02 00:15:00+00:00,2,HEATING,Bedroom,24,{'callForHeat': {'dataIntervals': [{'from': '2...,"{'dateCreated': '2020-10-18T13:33:17.982Z', 'd..."
3,2023-01-01,2022-12-31 23:45:00+00:00,2023-01-02 00:15:00+00:00,1,HEATING,Living room,24,{'callForHeat': {'dataIntervals': [{'from': '2...,"{'dateCreated': '2020-10-14T13:54:26.696Z', 'd..."
4,2023-01-01,2022-12-31 23:45:00+00:00,2023-01-02 00:15:00+00:00,6,HEATING,Haadiyah Room,24,{'callForHeat': {'dataIntervals': [{'from': '2...,"{'dateCreated': '2020-12-02T18:51:08.031Z', 'd..."
...,...,...,...,...,...,...,...,...,...
2905,2022-07-12,2022-07-11 22:45:00+00:00,2022-07-12 23:15:00+00:00,10,HEATING,Downstairs hallway,24,{'callForHeat': {'dataIntervals': [{'from': '2...,"{'dateCreated': '2020-12-02T23:20:51.490Z', 'd..."
2906,2022-07-12,2022-07-11 22:45:00+00:00,2022-07-12 23:15:00+00:00,9,HEATING,Office,24,{'callForHeat': {'dataIntervals': [{'from': '2...,"{'dateCreated': '2020-12-02T20:05:04.960Z', 'd..."
2907,2022-07-12,2022-07-11 22:45:00+00:00,2022-07-12 23:15:00+00:00,12,HEATING,Upstairs hallway,24,{'callForHeat': {'dataIntervals': [{'from': '2...,"{'dateCreated': '2020-12-25T09:39:10.393Z', 'd..."
2908,2022-07-12,2022-07-11 22:45:00+00:00,2022-07-12 23:15:00+00:00,7,HEATING,Ensuite,24,{'callForHeat': {'dataIntervals': [{'from': '2...,"{'dateCreated': '2020-12-02T19:00:42.012Z', 'd..."


In [79]:
days_preprocessed = (
    days
    .drop(columns=['historic_data', 'zone_metadata', 't_start', 't_end'])
    .sort_values(by=['extracted_date', 'zone_id'])
    .assign(extracted_datetime=lambda x: pd.to_datetime(x.extracted_date, utc=True))
    .drop(columns=['extracted_date'])
    .astype({"zone_id": "Int8"})
)
days_preprocessed

,zone_id,zone_type,zone_name,hours_in_day,extracted_datetime
45,1,HEATING,Living room,24,2022-01-01 00:00:00+00:00
23,2,HEATING,Bedroom,24,2022-01-01 00:00:00+00:00
46,6,HEATING,Haadiyah Room,24,2022-01-01 00:00:00+00:00
11,7,HEATING,Ensuite,24,2022-01-01 00:00:00+00:00
0,9,HEATING,Office,24,2022-01-01 00:00:00+00:00
...,...,...,...,...,...
7,9,HEATING,Office,24,2023-01-01 00:00:00+00:00
6,10,HEATING,Downstairs hallway,24,2023-01-01 00:00:00+00:00
8,12,HEATING,Upstairs hallway,24,2023-01-01 00:00:00+00:00
5,16,HEATING,Sufyaan room,24,2023-01-01 00:00:00+00:00


In [80]:
merged = (
    pd.merge_asof(
        left=int_heat_weather, right=days_preprocessed,
        left_index=True, right_on='extracted_datetime', 
        direction='forward', by='zone_id'
    )
    .ffill()
    .drop(columns=['extracted_date', 'extracted_datetime'])
) 

merged

,humidity,temperature,zone_id,value,weather,exterior_temp,zone_type,zone_name,hours_in_day
timestamp,,,,,,,,,
2022-01-01 00:00:00+00:00,0.500,20.00,1,NONE,UNKNOWN,NaN,HEATING,Living room,24
2022-01-01 00:00:00+00:00,0.500,20.00,2,NONE,UNKNOWN,NaN,HEATING,Bedroom,24
2022-01-01 00:00:00+00:00,0.500,20.00,6,NONE,UNKNOWN,NaN,HEATING,Haadiyah Room,24
2022-01-01 00:00:00+00:00,0.500,20.00,7,NONE,UNKNOWN,NaN,HEATING,Ensuite,24
2022-01-01 00:00:00+00:00,0.500,20.00,9,NONE,UNKNOWN,NaN,HEATING,Office,24
...,...,...,...,...,...,...,...,...,...
2023-01-01 00:00:00+00:00,0.585,20.36,9,HIGH,UNKNOWN,1.31,HEATING,Office,24
2023-01-01 00:00:00+00:00,0.597,22.44,10,NONE,UNKNOWN,1.31,HEATING,Downstairs hallway,24
2023-01-01 00:00:00+00:00,0.607,21.87,12,NONE,UNKNOWN,1.31,HEATING,Upstairs hallway,24


Dataset saved to disk by the python module version of this preprocessing pipeline